In [1]:
import tensorflow as tf
import numpy as np

In [13]:
n_inputs = 3 
n_neurons = 5

In [14]:

X0 = tf.placeholder(tf.float32, [None, n_inputs]) 
X1 = tf.placeholder(tf.float32, [None, n_inputs])

Wx = tf.Variable(tf.random_normal(shape=[n_inputs, n_neurons],dtype=tf.float32)) 
Wy = tf.Variable(tf.random_normal(shape=[n_neurons,n_neurons],dtype=tf.float32)) 
b = tf.Variable(tf.zeros([1, n_neurons], dtype=tf.float32))

Y0 = tf.tanh(tf.matmul(X0, Wx) + b)
Y1 = tf.tanh(tf.matmul(Y0, Wy) + tf.matmul(X1, Wx) + b)



In [15]:
# Mini-batch:        instance 0,instance 1,instance 2,instance 3
X0_batch = np.array([[0, 1, 2], [3, 4, 5], [6, 7, 8], [9, 0, 1]]) # t = 0 
X1_batch = np.array([[9, 8, 7], [0, 0, 0], [6, 5, 4], [3, 2, 1]]) # t = 1

init = tf.global_variables_initializer()

with tf.Session() as sess: 
    init.run()
    Y0_val, Y1_val = sess.run([Y0, Y1], feed_dict={X0: X0_batch, X1: X1_batch})
    print(Y0_val)
    print(Y1_val)
    


[[ 0.89879841  0.99072748  0.90153933 -0.92278874  0.41099289]
 [ 0.9999997   0.99989456  1.         -0.99999988 -0.99999595]
 [ 1.          0.99999881  1.         -1.         -1.        ]
 [ 0.99999952 -0.9999299   1.         -0.99996901 -1.        ]]
[[ 1.          0.99945593  1.         -1.         -1.        ]
 [-0.98404354  0.74714833 -0.58133227 -0.87728298  0.91720599]
 [ 1.          0.99207693  1.         -1.         -1.        ]
 [ 0.99998528 -0.05925031  1.         -0.99732852 -0.99999732]]


Static unroll the network

In [16]:
n_steps = 4
X = tf.placeholder(tf.float32, [None, n_steps, n_inputs]) 
X_seqs = tf.unstack(tf.transpose(X, perm=[1, 0, 2])) 
basic_cell = tf.contrib.rnn.BasicRNNCell(num_units=n_neurons, reuse=tf.AUTO_REUSE) 
output_seqs, states = tf.contrib.rnn.static_rnn(basic_cell, X_seqs, dtype=tf.float32) 
outputs = tf.transpose(tf.stack(output_seqs), perm=[1, 0, 2])

In [17]:

X_batch = np.array([ 
    #t=0        t=1
    [[0, 1, 2], [9, 8, 7]], # instance 0 
    [[3, 4, 5], [0, 0, 0]], # instance 1
    [[6, 7, 8], [6, 5, 4]], # instance 2
    [[9, 0, 1], [3, 2, 1]], # instance 3 
])

X_batch = np.random.randint(0, high=10, size=(4, n_steps, n_inputs))

init = tf.global_variables_initializer()

with tf.Session() as sess: 
    init.run()
    outputs_val = outputs.eval(feed_dict={X: X_batch})
    print(outputs_val.shape, outputs_val)

((4, 4, 5), array([[[ 0.86637133, -0.55519849, -0.97251904,  0.03270818, -0.91350502],
        [-0.99928916,  0.86048329,  0.54540539, -0.99984103,  0.97426826],
        [-0.99949807,  0.98524779,  0.98031551, -0.9987793 ,  0.98445666],
        [-0.97256231,  0.94887215, -0.42968053, -0.97392356, -0.24955133]],

       [[-0.99709272,  0.91775   ,  0.93292534, -0.99920511,  0.98479062],
        [-0.37246856,  0.81533486, -0.96785265, -0.25090876, -0.97511387],
        [ 0.92111909, -0.61718714, -0.99999309,  0.87952876, -0.9999662 ],
        [-0.05040411, -0.39962953, -0.98243755, -0.91505539, -0.75839227]],

       [[-0.93098873,  0.70039928, -0.76462626, -0.99884892,  0.36842662],
        [-0.99989593,  0.98114711,  0.97863096, -0.99963963,  0.98942637],
        [-0.99387884,  0.97967857,  0.51920283, -0.95552492,  0.28007409],
        [-0.92052209,  0.89755243, -0.97957492, -0.98184401, -0.92720497]],

       [[ 0.75067335,  0.55973136, -0.99999416, -0.10312977, -0.99997711],
       

Dynamic RNN

In [18]:
X = tf.placeholder(tf.float32, [None, n_steps, n_inputs]) 
basic_cell = tf.contrib.rnn.BasicRNNCell(num_units=n_neurons, reuse=True)
outputs, states = tf.nn.dynamic_rnn(basic_cell, X, dtype=tf.float32)

In [19]:

# X_batch = np.array([ 
#     #t=0        t=1
#     [[0, 1, 2], [9, 8, 7]], # instance 0 
#     [[3, 4, 5], [0, 0, 0]], # instance 1
#     [[6, 7, 8], [6, 5, 4]], # instance 2
#     [[9, 0, 1], [3, 2, 1]], # instance 3 
# ])
# X_batch = np.random.randint(0, high=10, size=(4, n_steps, n_inputs))

with tf.Session() as sess: 
    init.run()
    outputs_val = outputs.eval(feed_dict={X: X_batch})
    print(outputs_val.shape, outputs_val)

((4, 4, 5), array([[[ 0.63920832,  0.99621069, -0.99197495, -0.96988714,  0.51384735],
        [-0.99902743,  0.99999791, -0.65546632,  0.94535208, -0.94744056],
        [-0.93153596,  1.        , -0.7001242 ,  0.53310984,  0.0397525 ],
        [-0.94903445,  0.99999583, -0.97840238, -0.09500547, -0.7994203 ]],

       [[-0.96030819,  0.99999136, -0.01352669,  0.80969471, -0.46598232],
        [-0.65358245,  0.99960017, -0.96045309, -0.87274522, -0.64940554],
        [-0.87467957,  0.99291646, -0.99750835, -0.97636789, -0.95555443],
        [-0.9072178 ,  0.99981105, -0.96516299, -0.80722332, -0.70015013]],

       [[-0.82083577,  1.        , -0.99590266, -0.83910519, -0.00975125],
        [-0.99199474,  1.        , -0.91792309,  0.62127179, -0.56008416],
        [-0.97757614,  0.99955827, -0.17544271,  0.85564941, -0.89829391],
        [-0.9245609 ,  0.99999994, -0.99096477, -0.87989563, -0.67832375]],

       [[-0.96992928,  0.99739987, -0.97243834, -0.70799577, -0.99237132],
       

Variable Sequence Length

In [20]:
seq_length = tf.placeholder(tf.int32, [None])
basic_cell = tf.contrib.rnn.BasicRNNCell(num_units=n_neurons, reuse=True)
outputs, states = tf.nn.dynamic_rnn(basic_cell, X, dtype=tf.float32, sequence_length=seq_length)

# X_batch = np.array([
#     # step 0    step 1
#     [[0, 1, 2], [9, 8, 7]], # instance 0
#     [[3, 4, 5], [0, 0, 0]], # instance 1 (padded with a zero vector) 
#     [[6, 7, 8], [6, 5, 4]], # instance 2
#     [[9, 0, 1], [3, 2, 1]], # instance 3
# ])
seq_length_batch = np.array([2, 1, 2, 2])

init = tf.global_variables_initializer()

with tf.Session() as sess: 
    init.run()
    outputs_val, states_val = sess.run([outputs, states], feed_dict={X: X_batch, seq_length: seq_length_batch})
    print("-----------------------------------------------")
    print(outputs_val)
    print("-----------------------------------------------")
    print(states_val)

-----------------------------------------------
[[[-0.99922597 -0.96967816 -0.94536936  0.20033468  0.99993646]
  [-0.20636128  0.99975294 -0.99999988  0.9300521   0.99906361]
  [ 0.          0.          0.          0.          0.        ]
  [ 0.          0.          0.          0.          0.        ]]

 [[-0.77677733  0.99366647 -0.99999833  0.79104751  0.99872667]
  [ 0.          0.          0.          0.          0.        ]
  [ 0.          0.          0.          0.          0.        ]
  [ 0.          0.          0.          0.          0.        ]]

 [[-0.99994415  0.33028042 -0.99999994  0.88590157  1.        ]
  [-0.99626333  0.99961406 -1.         -0.06410188  0.99992627]
  [ 0.          0.          0.          0.          0.        ]
  [ 0.          0.          0.          0.          0.        ]]

 [[-0.14401278 -0.99762577 -0.9901731   0.99565089  0.99998021]
  [-0.8333298  -0.17101981 -0.99957359  0.90175283  0.99994987]
  [ 0.          0.          0.          0.        